# Relations in Database

This topic is basically and extention for session 13 in which we talked about databases. In real projects, we often encounter situations where we need to fetch data from multiple tables rather than one. This is where this topic come's in; we're to learn how we can properly connect database tables for out project.

There is basiaclly 3 types of relation between tables:
1. one to one
2. one to many
3. many to many

It's not possible to draw diagrams in Jupyter Notebook so I'll try to verbally explain what these mean. Imagine you're designing a database for a university LMS, it is obvious that each student can choose many subjects, and each subject can be chosen by many students. So, in case we have one table for subjects and one for students, these tables would be in *many to many* relation. One example of *one to many* relation would be instructors, each instructor can teach many subjects but each subject can be taught by one instructor. Can you think of any *one to one* relations in a university LMS database?

Back to our project (which is the same one from session 14). Since each product can be in one category and each category can have many products, we will implement a *one to many* relation for our `Product` model and a new model called `Category`. We'll start by making this new model.

In [ ]:
from django.db import models
from django.utils.text import slugify

class Category(models.Model):
    name = models.CharField(max_length=300, verbose_name="Category")
    url_title = models.CharField(max_length=300, verbose_name="url_title")

    def __str__(self):
        return self.name    

Then, we need to alter our `Product` class to be connected to `Category`. but first, we need to understand what a Foreign Key is.

You surely remember what Primary Key was. The record's identification which was mostly generated by the ORM itself. The Primary Key in our `Product` table was `id`. `Category` model will also have an `id` field which will be auto generated by Django ORM, you see where we are going? If we somehow show the `id` of target category in the `Product` table, the relation will be completed. 

For example, we have a record of a mobile phone "A" in our `Product` table. There is a record in `Category` table for Mobile Phone who's `id` is 10 (or which ever number). All we need to store in databse is a number 10 for that mobile phone "A" to know which category that product belongs too.

The field in `Product` model which stores `id`s from `Category` table is called a *Foreign Key*.

In [ ]:
class Product(models.Model):
    name = models.CharField(default="N/A", max_length=100, blank=False)
    price = models.IntegerField(default=0)
    is_active = models.BooleanField(default=True)
    slug = models.SlugField(default="", unique=True, blank=True)
    category = models.ForeignKey( # This is the foreign key
        Category, 
        on_delete=models.CASCADE,
        null=True # To keep our existing data safe
    ) 

You have probably noticed `on_delete=models.CASCADE`. This is telling Django that in case category "C" was deleted, delete all instances of that category from `Product` table too. One alternative here is `on_delete=models.PROTECT` which does exactly the opposite. And another would be `on_delete=models.SET_NULL` which keeps the record in the `Product` table but removes the data in the foreign key's column.

So, how should we use this relation to our benefit. Actually, accessing is very easy here. Let's say you've opened a Django Shell and want to play with database. You want to see the list of products along with their category name. Not category id, category name! Basically you want to see the related data in two separate tables. In SQL, we perform the join operation to resolve this. Django does that too, behind the scenes. But our work is way easier:

In [ ]:
products = Product.objects.all()

for p in products:
    print(p.name, p.category_id) 

This will select all related objects of Category table who have a reference in Product table. Note that we have overrided `__str__` method before so this code will automatically print all the names of those objects alongside their product name. If you were to get any other attribute from the Category table:

In [ ]:
products = Product.objects.all()

for p in products:
    print(p.name, p.category_id.url_title) # Or any other field.

The access here was from Product table to Category table, this can be the other way around, you can see the list of all products who have a category of "C". Even though you can simply use `filter()` with objects of Product table, the more standard way is to access them from Category table.

In [ ]:
watches = Category.objects.get(name="Smartwatch")

print(watches.product_set.all())

The name `product_set` is generated by Django behind the scene, it uses the name you gave to the target model to make this method. We used Product here so it's `product_set`. If we used, Khar, it would've been `khar_set`.

You can control what Django calls this method by altering the foreign key field in Product model:

In [ ]:
class Product(models.Model):
    name = models.CharField(default="N/A", max_length=100, blank=False)
    price = models.IntegerField(default=0)
    is_active = models.BooleanField(default=True)
    slug = models.SlugField(default="", unique=True, blank=True)
    categgory = models.ForeignKey( # This is the foreign key
        Category, 
        on_delete=models.CASCADE,
        null=True,
        related_name='khar' # it's now watches.khar.all()
    ) 

**Of course it's not needed to do the migrations thing again since you altered the model layer!**

As for the **One to One** relation, it's fairly simple to implement. As for it's applications though, Not so many examples. One significant example of this relation is for user table. For signing up your users, you only need their username, password and phone number. But there are a lot more fields in to store for each user, for example in their profile: bio, avatar, birthdate, etc. A one to one relations between the User and Profile table would be a good idea.

In [ ]:
from django.contrib.auth.models import User

class Profile(models.Model):
    user = models.OneToOneField(to=User, on_delete=models.CASCADE)

The rest is basically the same. Just One to One!

For many to many relations, we'll work this through a example. Imagine each of your products have a number of tags, and each tag is assigned to a number of products. This is a Many to Many relation. To implement this we'll do as follows:

In [ ]:
class Tag(models.Model):
    name = models.CharField(max_length=50)

class Product(models.Model):
    name = models.CharField(default="N/A", max_length=100, blank=False)
    price = models.IntegerField(default=0)
    is_active = models.BooleanField(default=True)
    slug = models.SlugField(default="", unique=True, blank=True)
    categgory = models.ForeignKey( # This is the foreign key
        Category, 
        on_delete=models.CASCADE,
        null=True,
        related_name='khar' # it's now watches.khar.all()
    )
    tag = models.ManyToManyField(Tag) # No on Delete here

There is one cool detail to mention about this type or relation. For the other two types, Django uses one of the sides of the relation to store the data of the connection. But for many to many relation, Django creates a third table that is in charge of storing data of the connection. This third table has two main colums, one a foreign key to product_id, and one a foreign key to tag_id. So if one tag is deleted, the wouldn't delete, or vice versa. This errors that might happen in database level, like deleting that result in unintended loss of data, or creating that results in unintender creation of data; are called *anomalies*. Which Django is the best at preventing them. 

Ok so how to use this type of relation? We can do the following in shell or essentially anyware in our app.

In [ ]:
target = Product.objects.get("some product")
tag = Tag.objects.create(name="tag_name")
target.tag.add(tag)

Very simple. Something to keep in mind though is that you must not assign anything to `target.tag` since it's actually a list of things; eventually would cause the database to break.

Another way of accessing is to see all the products who have some tag!

In [ ]:
tag = Tag.objects.get("some tag")
tag_prods = tag.product_set.all()

Easy Peasy!

And that is pretty much it for database relations. Of course the potential of things you can do with these relations are limitless. I trust you to be a good learned and take a look at Django Documentation if you want a deep undersanding of these relations. Either now or whenerver you needed this deep knowledge.